In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from pathlib import Path
import json
import glob
import numpy as np
import librosa
import os
from librosa import display
import matplotlib.pyplot as plt
from DLNet_functions import PreprocessWrapper
from models import ModelType, ModelBuilder
import tensorflow as tf
from time import strftime
assert tf.__version__ >= "2.0"
# autotune computation
AUTOTUNE = tf.data.experimental.AUTOTUNE
DATA_PATH = '/content/drive/MyDrive'

In [10]:
 # Read config from dataset:
json_file = os.path.join(DATA_PATH, 'tf_dataset_uncompr_wav', 'DLNet_config.json')
with open(json_file, "r") as read_file:
            config = json.load(read_file)
# Path to dataset_config:
ds_config: str = os.path.join(DATA_PATH, 'dataset_config.json')
# Creater wrapper object:
wrapper: PreprocessWrapper = PreprocessWrapper(config, ds_config)
# Print Config:
config

{'audio_length': 1,
 'binary': True,
 'calculate_mel': False,
 'center': True,
 'classes': ['compressed_wav', 'uncompr_wav'],
 'filter_signal': False,
 'hop_length': 256,
 'input_shape': [513, 173, 1],
 'mono': True,
 'n_fft': 1024,
 'n_frames': 173,
 'n_mels': 64,
 'pad_mode': 'reflect',
 'power': 2.0,
 'random_seed': 10,
 'sr': 44100,
 'time_stamp': '14_02_2021_13_23',
 'win_length': 512,
 'window': 'hamm'}

In [11]:
# Path to datasets:
# AAC
path_aac_train = os.path.join(DATA_PATH, 'tf_dataset_aac_128', 'aac_128_train_set')
path_aac_test = os.path.join(DATA_PATH, 'tf_dataset_aac_128', 'aac_128_train_set')

# MP3 32k
path_mp3_32k_train = os.path.join(DATA_PATH, 'tf_dataset_mp3_32k', 'mp3_32k_train_set')
path_mp3_32k_test = os.path.join(DATA_PATH, 'tf_dataset_mp3_32k', 'mp3_32k_train_set')

# MP3 160k
path_mp3_160k_train = os.path.join(DATA_PATH, 'tf_dataset_mp3_160k', 'mp3_160k_train_set')
path_mp3_160k_test = os.path.join(DATA_PATH, 'tf_dataset_mp3_160k', 'mp3_160k_train_set')

# MP3 192k
path_mp3_192k_train = os.path.join(DATA_PATH, 'tf_dataset_mp3_192k', 'mp3_192k_train_set')
path_mp3_192k_test = os.path.join(DATA_PATH, 'tf_dataset_mp3_192k', 'mp3_192k_train_set')

# MP3 320k
path_mp3_320k_train = os.path.join(DATA_PATH, 'tf_dataset_mp3_320k', 'mp3_320k_train_set')
path_mp3_320k_test = os.path.join(DATA_PATH, 'tf_dataset_mp3_320k', 'mp3_320k_train_set')

# AC3
path_ogg_vbr_train = os.path.join(DATA_PATH, 'tf_dataset_ogg_vbr', 'ogg_vbr_train_set')
path_ogg_vbr_test = os.path.join(DATA_PATH, 'tf_dataset_ogg_vbr', 'ogg_vbr_train_set')

# Uncompressed/Lossless
path_uncompr_train = os.path.join(DATA_PATH, 'tf_dataset_uncompr_wav', 'uncompr_wav_train_set')
path_uncompr_test = os.path.join(DATA_PATH, 'tf_dataset_uncompr_wav', 'uncompr_wav_train_set')

In [18]:
# Load dataset:
# Compressed
train_compr = wrapper.load_tf_dataset(path_aac_train)
test_compr = wrapper.load_tf_dataset(path_aac_test)

# Uncompressed/Lossless
train_uncompr = wrapper.load_tf_dataset(path_uncompr_train)
test_uncompr = wrapper.load_tf_dataset(path_uncompr_test)

# Concatenate datasets:
train_dataset = train_uncompr.concatenate(train_compr)
test_dataset = test_uncompr.concatenate(test_compr)


In [19]:
train_dataset

<ConcatenateDataset shapes: ((513, 173, 1), (2,)), types: (tf.float32, tf.uint8)>

In [20]:
train_size = len(train_dataset)
test_size = len(test_dataset)
eval_size = int(.1*train_size)
batch_size = 64

# Shuffel train data:
train_dataset = train_dataset.shuffle(buffer_size=train_size)

# Split train into train and eval set:
eval_dataset = train_dataset.take(eval_size)
eval_dataset = eval_dataset.batch(batch_size).prefetch(AUTOTUNE)

# Train dataset
train_dataset = train_dataset.skip(eval_size)
train_dataset = train_dataset.shuffle(train_size - eval_size)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(AUTOTUNE)

# Prepare test dataset
test_dataset = test_dataset.batch(batch_size).prefetch(AUTOTUNE)


In [21]:
train_dataset

<PrefetchDataset shapes: ((None, 513, 173, 1), (None, 2)), types: (tf.float32, tf.uint8)>

In [16]:
MODEL_TYPE = ModelType.HENNEQUIN
model_builder = ModelBuilder(MODEL_TYPE, config['input_shape'],
                             config['classes'])
model = model_builder.get_model()

# Define metrics
metrics = [tf.keras.metrics.TrueNegatives(),
           tf.keras.metrics.TruePositives(),
           tf.keras.metrics.FalseNegatives(),
           tf.keras.metrics.FalsePositives(),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
           tf.keras.metrics.CategoricalAccuracy()
           ]

In [22]:
# compile model
n_epochs = 2
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# fit model
history = model.fit(train_dataset, epochs=n_epochs,
                    validation_data=eval_dataset)

model.evaluate(test_dataset, batch_size=64)

Epoch 1/2


NotFoundError: ignored

In [ ]:
# setup plot
fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(16,4))

# plot loss
ax[0].plot(range(n_epochs), history.history['loss'])
ax[0].plot(range(n_epochs), history.history['val_loss'])
ax[0].set_ylabel('loss'), ax[0].set_title('train_loss vs val_loss')

# plot accuracy
ax[1].plot(range(n_epochs), history.history['categorical_accuracy'])
ax[1].plot(range(n_epochs), history.history['val_categorical_accuracy'])
ax[1].set_ylabel('accuracy'), ax[1].set_title('train_acc vs val_acc')

# plot adjustement
for a in ax:
    a.grid(True)
    a.legend(['train','val'], loc=4)
    a.set_xlabel('num of Epochs')
plt.show()
